# Agents

In this notebook, we will learn how to create and configure Agents using the Amazon Bedrock API.

An Agent is an instance that combines a language model with automated actions, business rules, and, optionally, integration with Knowledge Bases (KB) or Flows, allowing you to build smarter and more dynamic conversational experiences.

Notebook contents:

- Initialising the Bedrock client and configuring the environment.

- Creating an Agent and defining aliases.

- Testing basic interaction with the Agent.

In [10]:
import time
import boto3
import logging
import uuid
import json

from utils import invoke_agent_helper

In [9]:
iam_client = boto3.client('iam')
sts_client = boto3.client('sts')
session = boto3.session.Session()
region = session.region_name

account_id = sts_client.get_caller_identity()["Account"]
bedrock_agent_client = boto3.client('bedrock-agent')
bedrock_agent_runtime_client = boto3.client('bedrock-agent-runtime')

In [18]:
lambda_function_arn = "arn:aws:lambda:xx-xxxxx-s:XXXXXXXX:function:lambda-sns"
lambda_function_name = lambda_function_arn.split(':')[-1]

model_sonnet_id = "eu.anthropic.claude-3-7-sonnet-20250219-v1:0"

# Create Agent

In [2]:
# define agent params
agent_name = 'notifier-agent'

agent_description = "Agent to test"
agent_instruction = "You are a notifier agent, helping clients to notifier information"
agent_action_group_description = "Actions for send sns notifier"

agent_action_group_name = "SNSNotifier"

First, we must create a role and add the necessary policies.

In [11]:
# create agent role
agent_role = "arn:aws:iam::XXXXXXXXXX:role/AmazonBedrockExecutionRoleForAgents_notifier-agent"

Once the role has been created, we can now create the agent.

In [ ]:
response = bedrock_agent_client.create_agent(
    agentName=agent_name,
    agentResourceRoleArn=agent_role,
    description=agent_description,
    idleSessionTTLInSeconds=1800,
    foundationModel=model_sonnet_id,
    instruction=agent_instruction,
)
response

Just like kb, we have an agent identifier. With this identifier, we can associate the agent with the action, kb, etc.

In [ ]:
agent_id = response['agent']['agentId']
agent_id

## Agent Action Group

In [17]:
# define agent functions
agent_functions = [
    {
        'name': 'notifier_sns',
        'description': 'Send an email to user',
        'parameters': {
            "message_id": {
                "description": "The message title",
                "required": True,
                "type": "string"
            }
        }
    }]

In [19]:
# configure and create an action group
agent_action_group_response = bedrock_agent_client.create_agent_action_group(
    agentId=agent_id,
    agentVersion='DRAFT',
    actionGroupExecutor={
        'lambda': lambda_function_arn
    },
    actionGroupName=agent_action_group_name,
    functionSchema={
        'functions': agent_functions
    },
    description=agent_action_group_description
)

In [ ]:
agent_action_group_response

In [21]:
# Create allow to invoke permission on lambda
lambda_client = boto3.client('lambda')

response = lambda_client.add_permission(
        FunctionName=lambda_function_name,
        StatementId=f'allow_bedrock_{agent_id}',
        Action='lambda:InvokeFunction',
        Principal='bedrock.amazonaws.com',
        SourceArn=f"arn:aws:bedrock:{region}:{account_id}:agent/{agent_id}",
    )

# Activate Agent

We have now defined our agent and its associated action.
- Finish associating the IAM permissions
- Now we can prepare and invoke it.

In [ ]:
response = bedrock_agent_client.prepare_agent(
    agentId=agent_id
)
response

In [25]:
alias_id = 'TSTALIASID' # it works with any agent

In [38]:
session_id = str(uuid.uuid1())
query = "Hi, I am Emma. I want to be notifier about the time."

In [ ]:
response = invoke_agent_helper(query, session_id, agent_id, alias_id)
response

In [ ]:
invoke_agent_helper("Claro. 98861", session_id, agent_id, alias_id)